In [1]:
import pandas as pd
import numpy as np
import joblib 
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder # Example scalers/encoders
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score, accuracy_score, roc_auc_score, roc_curve, auc

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter


In [ ]:
PATH_TO_MODELS = 'D:/Do_an_tot_nghiep/apt-detection/ai_model/dataset/working2/'
RF_MODEL_FILE = 'random_forest_model.pkl'
XGB_MODEL_FILE = 'xgboost_model.pkl'
PATH_TO_FILES = 'D:/Do_an_tot_nghiep/apt-detection/ai_model/dataset/DAPT-2020/'
DAPT2020_DATA_FILE = 'merged_cleaned.csv' 
SCALER_FILE = 'scaler.pkl'

LABEL_COLUMN = 'Stage'

APT_LABELS_IN_DAPT2020 = ['Lateral Movement', 'Reconnaissance', 'Establish Foothold', 'Data Exfiltration']
BENIGN_LABELS_IN_DAPT2020 = ['Benign', 'BENIGN']

FEATURE_COLUMNS = [
    'Dst Port',
    'Flow Duration',
    'Tot Fwd Pkts',
    'Tot Bwd Pkts',
    'TotLen Fwd Pkts',
    'TotLen Bwd Pkts',
    'Fwd Pkt Len Max',
    'Fwd Pkt Len Min',
    'Fwd Pkt Len Mean',
    'Fwd Pkt Len Std',
    'Bwd Pkt Len Max',
    'Bwd Pkt Len Min',
    'Bwd Pkt Len Mean',
    'Bwd Pkt Len Std',
    'Flow Byts/s',
    'Flow Pkts/s',
    'Flow IAT Mean',
    'Flow IAT Std',
    'Flow IAT Max',
    'Flow IAT Min',
    'Fwd IAT Tot',
    'Fwd IAT Mean',
    'Fwd IAT Std',
    'Fwd IAT Max',
    'Fwd IAT Min',
    'Bwd IAT Tot',
    'Bwd IAT Mean',
    'Bwd IAT Std',
    'Bwd IAT Max',
    'Bwd IAT Min',
    'Fwd PSH Flags',
    'Fwd URG Flags',
    'Fwd Header Len',
    'Bwd Header Len',
    'Fwd Pkts/s',
    'Bwd Pkts/s',
    'Pkt Len Min',
    'Pkt Len Max',
    'Pkt Len Mean',
    'Pkt Len Std',
    'Pkt Len Var',
    'FIN Flag Cnt',
    'SYN Flag Cnt',
    'RST Flag Cnt',
    'PSH Flag Cnt',
    'ACK Flag Cnt',
    'URG Flag Cnt',
    'ECE Flag Cnt',
    'CWE Flag Count',
    'Down/Up Ratio',
    'Pkt Size Avg',
    'Fwd Seg Size Avg',
    'Bwd Seg Size Avg',
    'Subflow Fwd Pkts',
    'Subflow Fwd Byts',
    'Subflow Bwd Pkts',
    'Subflow Bwd Byts',
    'Init Fwd Win Byts',
    'Init Bwd Win Byts',
    'Fwd Act Data Pkts',
    'Fwd Seg Size Min',
    'Active Mean',
    'Active Std',
    'Active Max',
    'Active Min',
    'Idle Mean',
    'Idle Std',
    'Idle Max',
    'Idle Min',
    'Protocol_0',
    'Protocol_6',
    'Protocol_17',
]

NUMERICAL_FEATURES_TO_SCALE = [
    'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts',
    'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min',
    'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max',
    'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std',
    'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std',
    'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean',
    'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot',
    'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min',
    'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s',
    'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std',
    'Pkt Len Var', 'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg',
    'Bwd Seg Size Avg', 'Subflow Fwd Pkts', 'Subflow Fwd Byts',
    'Subflow Bwd Pkts', 'Subflow Bwd Byts', 'Init Fwd Win Byts',
    'Init Bwd Win Byts', 'Fwd Act Data Pkts', 'Fwd Seg Size Min',
    'Active Mean', 'Active Std', 'Active Max', 'Active Min',
    'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min'
]

FEATURES_NOT_SCALED_BUT_NUMERICAL = [
    'Dst Port', # Cổng đích - có thể coi là số hoặc phân loại tùy cách dùng
    'Fwd PSH Flags', 'Fwd URG Flags', 'FIN Flag Cnt', 'SYN Flag Cnt',
    'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
    'CWE Flag Count', 'ECE Flag Cnt',
    'Protocol_0', 'Protocol_6', 'Protocol_17' # Kết quả của OHE Protocol
]

In [ ]:
import os
import joblib
# --- Load Mô hình và Scaler đã huấn luyện trên CIC-IDS2018 ---
print("Loading existing models and scaler...")
try:
    rf_model_cicids = joblib.load(os.path.join(PATH_TO_MODELS, RF_MODEL_FILE))
    xgb_model_cicids = joblib.load(os.path.join(PATH_TO_MODELS, XGB_MODEL_FILE))
    data_scaler = joblib.load(os.path.join(PATH_TO_MODELS, SCALER_FILE)) # Load your scaler here
    print("Models and scaler loaded successfully.")
    # Optional: print some info about loaded models/scaler
    print("RF Model Params:", rf_model_cicids.get_params())
    print("XGB Model Params:", xgb_model_cicids.get_params())
    print("Scaler object type:", type(data_scaler)) # Should be StandardScaler, MinMaxScaler, ColumnTransformer, etc.
except FileNotFoundError as e:
    print(f"Error loading required files: {e}")
    print("Please check the file paths and names for models and scaler.")
    exit() # Exit if files can't be loaded
except Exception as e:
     print(f"Error loading .pkl files: {e}")
     exit()


# --- Load Dữ liệu DAPT2020 ---
print(f"Loading DAPT2020 data from {os.path.join(PATH_TO_FILES, DAPT2020_DATA_FILE)}...")
try:
    # TODO: Kiểm tra encoding của file CSV DAPT2020 nếu gặp lỗi đọc file
    df_dapt2020 = pd.read_csv(os.path.join(PATH_TO_FILES, DAPT2020_DATA_FILE))
    print(f"DAPT2020 data loaded. Shape: {df_dapt2020.shape}")
    # Optional: Display first few rows and info
    print(df_dapt2020.head())
    print(df_dapt2020.info())
except FileNotFoundError as e:
    print(f"Error loading DAPT2020 data file: {e}")
    print("Please check the file path.")
    exit()
except Exception as e:
     print(f"Error reading DAPT2020 data file: {e}")
     exit()

Loading existing models and scaler...
Models and scaler loaded successfully.
Loading DAPT2020 data from D:/Do_an_tot_nghiep/apt-detection/ai_model/dataset/DAPT-2020/merged_cleaned.csv...
DAPT2020 data loaded. Shape: (165154, 85)
                                  Flow ID        Src IP  Src Port  \
0                   8.0.6.4-8.6.0.1-0-0-0       8.6.0.1         0   
1  192.168.3.10-239.2.11.71-53569-8662-17  192.168.3.10     53569   
2        255.255.255.255-0.0.0.0-67-68-17       0.0.0.0        68   
3  192.168.3.30-192.168.3.31-40504-9200-6  192.168.3.30     40504   
4              0.87.248.248-3.0.0.0-0-0-0  0.87.248.248         0   

            Dst IP  Dst Port  Protocol               Timestamp  Flow Duration  \
0          8.0.6.4         0         0  15/07/2019 01:55:21 PM      119998944   
1      239.2.11.71      8662        17  15/07/2019 01:55:22 PM      109235816   
2  255.255.255.255        67        17  15/07/2019 01:55:22 PM      119764062   
3     192.168.3.31      9200    

In [4]:
# 1. Xử lý các giá trị không xác định/thiếu và vô hạn trong DAPT2020
# TODO: ÁP DỤNG CÁCH XỬ LÝ GIÁ TRỊ THIẾU NHẤT QUÁN VỚI CIC-IDS2018
print("Preprocessing DAPT2020 data (handling NaNs and infs)...")
# Example: fill NaNs with 0 (adjust if needed)
df_dapt2020.fillna(0, inplace=True)
# Handle potential infinite values
df_dapt2020.replace([np.inf, -np.inf], np.nan, inplace=True)
df_dapt2020.fillna(0, inplace=True) # Fill NaNs created by replacing inf
print("NaNs and infs handled.")

Preprocessing DAPT2020 data (handling NaNs and infs)...
NaNs and infs handled.


In [ ]:

# 2. Ánh xạ nhãn gốc sang nhãn Benign (0) và APT (1)
print("\nMapping DAPT2020 labels to Benign/APT...")
if LABEL_COLUMN not in df_dapt2020.columns:
    print(f"Error: Label column '{LABEL_COLUMN}' not found in DAPT2020 data.")
    print("Please check the LABEL_COLUMN definition.")
    exit()

df_dapt2020['APT_Label'] = 0 # Default to Benign (0)
df_dapt2020.loc[df_dapt2020[LABEL_COLUMN].isin(APT_LABELS_IN_DAPT2020), 'APT_Label'] = 1 # Mark APT (1)
print(f"Label mapping done. Distribution: {Counter(df_dapt2020['APT_Label'])}")


# --- Thêm các cột đặc trưng bị thiếu vào DAPT2020 và điền giá trị 0 ---
print("\nChecking for missing feature columns in DAPT2020...")
missing_feature_columns = [col for col in FEATURE_COLUMNS if col not in df_dapt2020.columns]

if missing_feature_columns:
    print(f"Missing original feature columns found: {missing_feature_columns}")
    print("Adding missing columns to DAPT2020 and filling with 0...")
    for col in missing_feature_columns:
        df_dapt2020[col] = 0 # Add the column and fill with 0
    print("Missing columns added.")
else:
    print("All original feature columns are present in DAPT2020.")

# --- Sắp xếp lại các cột của DAPT2020 theo thứ tự của FEATURE_COLUMNS ---
print("\nReordering DAPT2020 columns to match original feature order...")
try:
    # Lấy danh sách các cột không phải là đặc trưng và không phải là nhãn mới tạo
    other_cols = [col for col in df_dapt2020.columns if col not in FEATURE_COLUMNS and col != 'APT_Label']
    # Sắp xếp lại: đặc trưng trước, nhãn mới, rồi đến các cột khác
    df_dapt2020 = df_dapt2020[FEATURE_COLUMNS + ['APT_Label'] + other_cols].copy()
    print(f"Columns reordered. New shape: {df_dapt2020.shape}")
except KeyError as e:
    print(f"Error reordering columns: {e}. Ensure all columns in FEATURE_COLUMNS exist after adding missing ones.")
    exit()
except Exception as e:
    print(f"An unexpected error occurred during column reordering: {e}")
    exit()

# --- Tách đặc trưng (X) và nhãn (y) ---
print("\nSeparating features (X) and labels (y)...")
try:
    # Chọn các cột đặc trưng THEO ĐÚNG THỨ TỰ trong FEATURE_COLUMNS
    X_dapt = df_dapt2020[FEATURE_COLUMNS].copy()
    y_dapt = df_dapt2020['APT_Label'].copy()
    print(f"Features shape (X_dapt): {X_dapt.shape}")
    print(f"Labels shape (y_dapt): {y_dapt.shape}")
    # Kiểm tra xem số cột của X_dapt có khớp với FEATURE_COLUMNS không
    if X_dapt.shape[1] != len(FEATURE_COLUMNS):
         print(f"WARNING: X_dapt has {X_dapt.shape[1]} columns, but FEATURE_COLUMNS has {len(FEATURE_COLUMNS)} items.")
except KeyError as e:
    print(f"Error selecting columns for X_dapt or y_dapt: {e}")
    print("Please ensure FEATURE_COLUMNS and 'APT_Label' exist after reordering.")
    exit()
except Exception as e:
    print(f"An unexpected error occurred during X/y separation: {e}")
    exit()

# --- TIỀN XỬ LÝ VÀ LÀM SẠCH X_dapt TRƯỚC KHI SCALING ---
print("\n--- Pre-scaling Data Cleaning for X_dapt ---")

# 1. Chuyển đổi kiểu dữ liệu và xử lý lỗi tiềm ẩn
print("Attempting data type conversion (to numeric where possible)...")
potential_object_cols = X_dapt.select_dtypes(include=['object']).columns.tolist()
if potential_object_cols:
    print(f"Columns with object dtype found: {potential_object_cols}. Attempting pd.to_numeric...")
    for col in potential_object_cols:
        original_dtype = X_dapt[col].dtype
        try:
            # Cố gắng chuyển đổi, lỗi sẽ thành NaN
            X_dapt[col] = pd.to_numeric(X_dapt[col], errors='coerce')
            if X_dapt[col].isnull().any():
                print(f"  - Column '{col}': Conversion done, some values became NaN.")
            # else:
            #     print(f"  - Column '{col}': Conversion successful.")
        except Exception as conv_err:
            print(f"  - Error converting column '{col}' (original dtype {original_dtype}): {conv_err}")
            # Quyết định xem có nên dừng lại nếu chuyển đổi thất bại không
            # exit()
else:
    print("No object columns found in features.")

# 2. Xử lý giá trị Vô cực (Infinity)
print("Checking for and replacing Infinity values...")
numeric_cols_check = X_dapt.select_dtypes(include=np.number)
inf_check = np.isinf(numeric_cols_check).any().any() # Kiểm tra nhanh hơn
if inf_check:
    print("Infinity values found. Replacing with NaN...")
    # Chỉ thay thế trên các cột số để tránh lỗi dtype
    for col in numeric_cols_check.columns:
         # Kiểm tra từng cột để thay thế hiệu quả hơn
         if np.isinf(X_dapt[col]).any():
              X_dapt[col] = X_dapt[col].replace([np.inf, -np.inf], np.nan)
    print("Infinity replacement done.")
else:
    print("No Infinity values found.")

# 3. Xử lý giá trị Thiếu (NaN) - Sau khi chuyển đổi và thay thế Inf
print("Checking for and filling NaN values...")
nan_check = X_dapt.isnull().any().any()
if nan_check:
    nan_counts = X_dapt.isnull().sum()
    print(f"NaN values found. Total before fill: {nan_counts.sum()}")
    # print("NaN counts per column:\n", nan_counts[nan_counts > 0]) # In chi tiết nếu cần
    print("Filling NaN values with 0...") # Hoặc chiến lược khác: mean(), median()
    X_dapt = X_dapt.fillna(0)
    print("NaN filling done.")
else:
    print("No NaN values found.")

# 4. Kiểm tra cuối cùng trước khi scale
print("\nFinal check before scaling:")
print(f"X_dapt shape: {X_dapt.shape}")
final_nan_count = X_dapt.isnull().sum().sum()
final_inf_count = np.isinf(X_dapt.select_dtypes(include=np.number).to_numpy()).sum()
print(f"NaN count: {final_nan_count}")
print(f"Infinity count: {final_inf_count}")

# >>> KIỂM TRA QUAN TRỌNG NHẤT: Đảm bảo tất cả cột đặc trưng là số <<<
non_numeric_final = X_dapt.select_dtypes(exclude=[np.number]).columns.tolist()
if non_numeric_final:
    print(f"\nCRITICAL WARNING: Non-numeric columns still present in X_dapt after cleaning: {non_numeric_final}")
    print("These columns likely cause the scaler to fail. Please investigate!")
    # Xem xét việc dừng lại ở đây nếu scaler chắc chắn yêu cầu đầu vào số
    # exit()
else:
    print("All feature columns in X_dapt are now numeric.")

print("--- End Pre-scaling Data Cleaning ---")

# --- SO SÁNH FEATURE LISTS ---
print("\n--- Comparing Feature Lists ---")
# 1. Lấy danh sách features thực tế từ X_dapt
actual_features = X_dapt.columns.tolist()
print(f"Actual features in X_dapt ({len(actual_features)}):")
# In ra một phần để dễ nhìn
print(actual_features[:15], "..." if len(actual_features) > 15 else "")

# 2. Cố gắng lấy danh sách features mong đợi từ scaler
expected_features = None
expected_feature_count = None

if hasattr(data_scaler, 'feature_names_in_'):
    try:
        expected_features = list(data_scaler.feature_names_in_)
        expected_feature_count = len(expected_features)
        print(f"\nScaler EXPECTS features via 'feature_names_in_' ({expected_feature_count}):")
        print(expected_features[:15], "..." if len(expected_features) > 15 else "")

        # So sánh trực tiếp
        if actual_features == expected_features:
            print("\n[SUCCESS] Feature names and order MATCH perfectly!")
        else:
            print("\n[ERROR] Feature names and/or order MISMATCH!")
            if len(actual_features) != expected_feature_count:
                print(f"  - Length mismatch: Actual={len(actual_features)}, Expected={expected_feature_count}")
            # Tìm điểm khác biệt đầu tiên (ví dụ)
            for i, (actual, expected) in enumerate(zip(actual_features, expected_features)):
                if actual != expected:
                    print(f"  - First mismatch at index {i}: Actual='{actual}', Expected='{expected}'")
                    break
            else: # Nếu vòng lặp hoàn thành mà không break (chỉ xảy ra nếu độ dài khác nhau)
                if len(actual_features) > expected_feature_count:
                     print(f"  - Actual list has extra features starting from index {expected_feature_count}: '{actual_features[expected_feature_count]}'")
                elif len(actual_features) < expected_feature_count:
                     print(f"  - Expected list has extra features starting from index {len(actual_features)}: '{expected_features[len(actual_features)]}'")

    except Exception as e:
        print(f"\nCould not retrieve or process 'feature_names_in_': {e}")
        # Thử lấy số lượng features nếu có
        if hasattr(data_scaler, 'n_features_in_'):
            try:
                 expected_feature_count = data_scaler.n_features_in_
                 print(f"Scaler has 'n_features_in_': {expected_feature_count}")
                 if len(actual_features) == expected_feature_count:
                      print(f"[INFO] Number of features MATCHES ({expected_feature_count}), but names/order could not be verified via 'feature_names_in_'.")
                 else:
                      print(f"[ERROR] Number of features MISMATCH: Actual={len(actual_features)}, Expected={expected_feature_count}")
            except Exception as e2:
                 print(f"Could not retrieve 'n_features_in_': {e2}")
        else:
            print("Scaler does not have 'n_features_in_' attribute either.")

elif hasattr(data_scaler, 'n_features_in_'):
     try:
        expected_feature_count = data_scaler.n_features_in_
        print(f"\nScaler only has 'n_features_in_': {expected_feature_count}")
        if len(actual_features) == expected_feature_count:
             print(f"[INFO] Number of features MATCHES ({expected_feature_count}), but names/order cannot be verified automatically.")
             print("       Manual check needed based on training code or ColumnTransformer definition.")
        else:
             print(f"[ERROR] Number of features MISMATCH: Actual={len(actual_features)}, Expected={expected_feature_count}")
     except Exception as e:
         print(f"Could not retrieve 'n_features_in_': {e}")

else:
    print("\n[WARNING] Could not automatically retrieve expected features from the scaler object.")
    print("          Please manually verify feature names and order based on how the scaler was trained.")
    # Nếu là ColumnTransformer, cấu trúc của nó có thể giúp suy ra thứ tự
    if hasattr(data_scaler, 'transformers_'):
        print("          Since it might be a ColumnTransformer, inspect its 'transformers_' attribute:")
        try:
            inferred_order = []
            total_cols = 0
            for name, trans, cols in data_scaler.transformers_:
                 # 'cols' có thể là list tên hoặc list chỉ số
                 print(f"          - Step: '{name}', Columns specified: {cols[:15]}...")
                 # Chỉ thêm tên cột nếu nó là danh sách tên
                 if all(isinstance(c, str) for c in cols):
                     inferred_order.extend(cols)
                 total_cols += len(cols) # Đếm tổng số cột được xử lý
            print(f"          - Total columns processed by transformers: {total_cols}")
            # Lưu ý: Cách này không hoàn hảo nếu có cột bị drop hoặc remainder='passthrough'
        except Exception as inspect_err:
            print(f"          - Could not inspect 'transformers_': {inspect_err}")


print("--- End Comparing Feature Lists ---")

# --- Áp dụng scaler đã load ---
print("\nApplying loaded scaler to cleaned DAPT2020 features...")

# In thêm thông tin về scaler nếu là ColumnTransformer
if hasattr(data_scaler, 'transformers_'):
    print("\nInspecting Scaler (ColumnTransformer) Structure:")
    try:
        for name, trans, cols in data_scaler.transformers_:
            print(f"  - Step: '{name}', Transformer: {trans}, Columns: {cols[:15]}..." if len(cols)>15 else cols) # Rút gọn nếu quá dài
        if hasattr(data_scaler, 'feature_names_in_'):
             print(f"  - Scaler expected features ({len(data_scaler.feature_names_in_)}): {list(data_scaler.feature_names_in_)[:15]}...")
             print(f"  - Actual X_dapt columns ({len(X_dapt.columns)}): {X_dapt.columns.tolist()[:15]}...")
             if list(data_scaler.feature_names_in_) != X_dapt.columns.tolist():
                  print("  - WARNING: Actual column names/order DO NOT MATCH scaler's expected features!")
    except Exception as inspect_err:
        print(f"  - Could not fully inspect scaler structure: {inspect_err}")

try:
    # Gọi transform trên X_dapt ĐÃ ĐƯỢC LÀM SẠCH KỸ
    X_dapt_processed = data_scaler.transform(X_dapt)

    print(f"\nScaling successful!")
    print(f"Processed data shape after scaling: {X_dapt_processed.shape}")
    # Kiểm tra kiểu dữ liệu của output (thường là numpy array)
    print(f"Type of processed data: {type(X_dapt_processed)}")
    if isinstance(X_dapt_processed, np.ndarray):
         print(f"Processed data dtype: {X_dapt_processed.dtype}") # Thường là float64
         # Kiểm tra NaN/Inf trong kết quả cuối cùng (rất hiếm nhưng có thể)
         if np.isnan(X_dapt_processed).any() or np.isinf(X_dapt_processed).any():
              print("WARNING: NaN or Infinity found in the SCALED data!")

except TypeError as te: # Bắt cụ thể lỗi TypeError
     print(f"\nFATAL ERROR during scaler.transform: {te}")
     print("This 'TypeError' strongly suggests a data type mismatch.")
     print("Even after cleaning, some column(s) likely have a type incompatible with the scaler.")
     print("Possible Causes:")
     print("  1. A column intended for numeric scaling is still 'object' or another non-numeric type.")
     print("  2. The scaler (especially ColumnTransformer) definition mismatches X_dapt columns (names, order, or expected type).")
     print("  3. A specific transformer within the scaler failed on its input.")
     print("Recommendations:")
     print("  - Carefully review the 'CRITICAL WARNING' about non-numeric columns above.")
     print("  - Double-check the FEATURE_COLUMNS list and the column reordering step.")
     print("  - Inspect the scaler structure printed above - do the columns match?")
     print("  - Consider retraining the scaler on data preprocessed IDENTICALLY to DAPT2020.")
     import traceback
     print("\n--- Traceback ---")
     print(traceback.format_exc())
     print("--- End Traceback ---")
     exit() # Dừng lại vì không thể tiếp tục
except ValueError as ve:
     print(f"\nFATAL ERROR during scaler.transform: {ve}")
     print("This 'ValueError' often relates to incompatible values (e.g., NaN/Inf not handled) or incorrect number of features.")
     print("Recommendations:")
     print("  - Review the NaN/Inf counts printed just before scaling.")
     print("  - Ensure X_dapt has the exact number of columns the scaler expects.")
     import traceback
     print("\n--- Traceback ---")
     print(traceback.format_exc())
     print("--- End Traceback ---")
     exit()
except Exception as e:
    print(f"\nUNEXPECTED FATAL ERROR during scaler.transform: {e}")
    import traceback
    print("\n--- Traceback ---")
    print(traceback.format_exc())
    print("--- End Traceback ---")
    exit()

# --- Code tiếp theo (ví dụ: train_test_split) ---
# Chỉ chạy nếu X_dapt_processed được tạo thành công
if 'X_dapt_processed' in locals() and X_dapt_processed is not None:
    print("\nSplitting DAPT2020 data into train and test sets...")
    try:
        X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(
            X_dapt_processed,  # Dữ liệu đặc trưng đã được scale
            y_dapt,            # Nhãn (0 hoặc 1)
            test_size=0.2,     # Tỷ lệ dữ liệu cho tập kiểm tra (ví dụ: 20%)
            random_state=42,   # Để đảm bảo kết quả chia có thể lặp lại
            stratify=y_dapt    # Giữ nguyên tỷ lệ phân phối lớp
        )
        # ... (in kích thước và phân phối nhãn như code gốc) ...
        print(f"Shape of X_train_final: {X_train_final.shape}")
        print(f"Shape of X_test_final: {X_test_final.shape}")
        print(f"Train Label Distribution: {Counter(y_train_final)}")
        print(f"Test Label Distribution: {Counter(y_test_final)}")
        print("Data splitting complete.")

    except Exception as split_error:
        print(f"An error occurred during train_test_split: {split_error}")
else:
    print("\nSkipping train_test_split because data scaling failed or X_dapt_processed was not created.")



Mapping DAPT2020 labels to Benign/APT...
Label mapping done. Distribution: Counter({0: 123824, 1: 41330})

Checking for missing feature columns in DAPT2020...
Missing original feature columns found: ['CWE Flag Count', 'Protocol_0', 'Protocol_6', 'Protocol_17']
Adding missing columns to DAPT2020 and filling with 0...
Missing columns added.

Separating features (X) and labels (y)...
Features shape (X_dapt): (165154, 72)
Labels shape (y_dapt): (165154,)

--- Pre-scaling Data Cleaning for X_dapt ---
Attempting data type conversion (to numeric where possible)...
No object columns found in features.
Checking for and replacing Infinity values...
No Infinity values found.
Checking for and filling NaN values...
No NaN values found.

Final check before scaling:
X_dapt shape: (165154, 72)
NaN count: 0
Infinity count: 0
All feature columns in X_dapt are now numeric.
--- End Pre-scaling Data Cleaning ---

--- Comparing Feature Lists ---
Actual features in X_dapt (72):
['Dst Port', 'Flow Duration',

In [ ]:
# 3. Áp dụng scaler đã load cho các đặc trưng
# SỬ DỤNG SCALER ĐÃ LOAD TỪ CIC-IDS2018 ĐỂ TRANSFORM DỮ LIỆU DAPT2020
print("Applying loaded scaler to DAPT2020 features...")
try:
    X_dapt_processed = data_scaler.transform(X_dapt)
    print(f"Processed data shape after scaling: {X_dapt_processed.shape}")
    print("DAPT2020 preprocessing complete using loaded scaler.")
except Exception as e:
    print(f"Error applying the loaded scaler: {e}")
    print("Please ensure the loaded scaler is compatible with the shape and type of X_dapt.")
    print("Check the original preprocessing steps and the columns in FEATURE_COLUMNS.")
    exit()

# --- Chia tập dữ liệu ---
print("Splitting DAPT2020 data into train and test sets...")
X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(
    X_dapt_processed, y_dapt,
    test_size=0.2,      # 20% for testing (adjust as needed)
    random_state=42,    # for reproducibility
    stratify=y_dapt     # Stratify to maintain class distribution (crucial for imbalance)
)
print(f"Train/Test shapes: {X_train_final.shape} / {X_test_final.shape}")
print(f"Train Label Distribution: {Counter(y_train_final)}")
print(f"Test Label Distribution: {Counter(y_test_final)}")


# --- Xử lý mất cân bằng lớp (trên tập huấn luyện) ---
# Tấn công APT (nhãn 1) có thể rất ít trong tập huấn luyện.
# Có nhiều cách xử lý:
# 1. Dùng class_weight trong mô hình (đối với RF)
# 2. Dùng scale_pos_weight trong mô hình (đối với XGBoost binary classification)
# 3. Áp dụng kỹ thuật lấy mẫu (sampling) như SMOTE, Undersampling (sử dụng thư viện imblearn)
print("Handling class imbalance...")

# Phương án 1/2: Sử dụng trọng số lớp tích hợp trong mô hình
# Tính toán trọng số lớp cho RF (sử dụng 'balanced' hoặc dict tùy chọn)
# from sklearn.utils.class_weight import compute_class_weight
# classes = np.unique(y_train_final)
# weights = compute_class_weight('balanced', classes=classes, y=y_train_final)
# class_weights_dict = dict(zip(classes, weights))
# print("Computed class weights for RF:", class_weights_dict) # Pass this dict to RF's fit method or use 'balanced'

# Tính toán scale_pos_weight cho XGBoost (phân loại nhị phân 0/1)
apt_count_train = sum(y_train_final == 1)
benign_count_train = sum(y_train_final == 0)
# Tránh chia cho 0 nếu không có mẫu APT trong tập huấn luyện (dù stratify cố gắng đảm bảo)
scale_pos_weight_value = benign_count_train / apt_count_train if apt_count_train > 0 else 1
print(f"Computed scale_pos_weight for XGBoost: {scale_pos_weight_value}") # Pass this value to XGBoost's fit method or set as parameter


# Phương án 3 (Sử dụng imblearn - cần cài đặt `pip install imbalanced-learn`)
print("Applying SMOTE on training data...")
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_final, y_train_final)
print(f"Resampled train shape: {X_train_resampled.shape}")
print(f"Resampled train label distribution: {Counter(y_train_resampled)}")
# Use X_train_resampled, y_train_resampled for fitting instead of X_train_final, y_train_final


# --- Huấn luyện lại Mô hình ---
print("Retraining models on the new data...")

# Retrain Random Forest
# Sử dụng lại cấu trúc/siêu tham số từ mô hình CIC-IDS2018 hoặc điều chỉnh nếu cần
# Bạn có thể tạo lại đối tượng RF hoặc sử dụng lại rf_model_cicids và gọi .fit()
# Để sử dụng lại rf_model_cicids và các siêu tham số cũ:
rf_model_retrained = rf_model_cicids
# Hoặc tạo mới với các siêu tham số mong muốn, CẦN XỬ LÝ class_weight ở đây hoặc khi tạo đối tượng
# from sklearn.ensemble import RandomForestClassifier
# rf_model_retrained = RandomForestClassifier(n_estimators=..., max_depth=..., random_state=42, class_weight='balanced' # or pass class_weights_dict)

# Fit using the training data (original or resampled)
# If using class_weight='balanced' in constructor, no need for sample_weight here
rf_model_retrained.fit(X_train_final, y_train_final)
# If using class_weight dict:
# rf_model_retrained.fit(X_train_final, y_train_final, sample_weight=np.array([class_weights_dict[label] for label in y_train_final]))

print("Random Forest retraining complete.")

# Retrain XGBoost
# Sử dụng lại cấu trúc/siêu tham số từ mô hình CIC-IDS2018 hoặc điều chỉnh nếu cần
# Bạn có thể tạo lại đối tượng XGBoost hoặc sử dụng lại xgb_model_cicids và gọi .fit()
# Để sử dụng lại xgb_model_cicids và các siêu tham số cũ:
xgb_model_retrained = xgb_model_cicids
# Hoặc tạo mới với các siêu tham số mong muốn. CẦN XỬ LỬ scale_pos_weight ở đây hoặc khi tạo đối tượng
# import xgboost as xgb
# xgb_model_retrained = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss', use_label_encoder=False, scale_pos_weight=scale_pos_weight_value, # other params)


# Fit using the training data (original or resampled)
# Pass scale_pos_weight for binary classification with imbalance, unless set in constructor
xgb_model_retrained.fit(X_train_final, y_train_final, scale_pos_weight=scale_pos_weight_value)

print("XGBoost retraining complete.")


# --- Đánh giá Mô hình mới ---
print("\nEvaluating retrained models on the test set...")

models_to_evaluate = {
    "Random Forest (Retrained)": rf_model_retrained,
    "XGBoost (Retrained)": xgb_model_retrained
}

for name, model in models_to_evaluate.items():
    print(f"\n--- Evaluation for {name} ---")

    # Dự đoán trên tập kiểm tra
    y_pred = model.predict(X_test_final)
    # Dự đoán xác suất (để tính AUC)
    if hasattr(model, "predict_proba"):
       y_pred_proba = model.predict_proba(X_test_final)[:, 1] # Probability of the positive class (APT=1)
    else:
       y_pred_proba = [0] * len(y_test_final) # Placeholder if no predict_proba


    # In báo cáo phân loại chi tiết (Precision, Recall, F1-score)
    print("Classification Report:")
    labels = [0, 1]
    target_names = ['Benign', 'APT']
    print(classification_report(y_test_final, y_pred, labels=labels, target_names=target_names, zero_division=0))


    # In Ma trận nhầm lẫn
    print("Confusion Matrix:")
    print(confusion_matrix(y_test_final, y_pred, labels=labels)) # Ensure order of labels

    # Tính và in các metrics quan trọng cho dữ liệu mất cân bằng
    accuracy = accuracy_score(y_test_final, y_pred)
    precision = precision_score(y_test_final, y_pred, pos_label=1, zero_division=0) # Precision for APT (positive class)
    recall = recall_score(y_test_final, y_pred, pos_label=1, zero_division=0)       # Recall for APT (positive class)
    f1 = f1_score(y_test_final, y_pred, pos_label=1, zero_division=0)         # F1-score for APT (positive class)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision (APT=1): {precision:.4f}")
    print(f"Recall (APT=1): {recall:.4f}")
    print(f"F1-score (APT=1): {f1:.4f}")

    # Tính và in AUC-ROC
    try:
        if len(np.unique(y_test_final)) > 1:
            roc_auc = roc_auc_score(y_test_final, y_pred_proba)
            print(f"AUC-ROC: {roc_auc:.4f}")
        else:
             print("AUC-ROC cannot be calculated as only one class is present in test labels.")
    except Exception as e:
        print(f"Could not calculate AUC-ROC: {e}")


# --- Lưu Mô hình đã huấn luyện lại ---
print("\nSaving retrained models...")
NEW_RF_MODEL_FILE = 'rf_dapt2020_apt_retrained.pkl'
NEW_XGB_MODEL_FILE = 'xgb_dapt2020_apt_retrained.pkl'

try:
    joblib.dump(rf_model_retrained, os.path.join(PATH_TO_FILES, NEW_RF_MODEL_FILE))
    joblib.dump(xgb_model_retrained, os.path.join(PATH_TO_FILES, NEW_XGB_MODEL_FILE))
    print(f"Retrained models saved as {NEW_RF_MODEL_FILE} and {NEW_XGB_MODEL_FILE}")
except Exception as e:
    print(f"Error saving models: {e}")

print("\nFine-tuning process (retraining) complete.")

Applying loaded scaler to DAPT2020 features...
Error applying the loaded scaler: The feature names should match those that were passed during fit.
Feature names must be in the same order as they were in fit.

Please ensure the loaded scaler is compatible with the shape and type of X_dapt.
Check the original preprocessing steps and the columns in FEATURE_COLUMNS.
Splitting DAPT2020 data into train and test sets...


NameError: name 'X_dapt_processed' is not defined

: 